In [1]:
import numpy as np

**Exercises 1, 2, 3, 4, 5, and 6**

In [2]:
class Linear_Opt_Problem:
    
    def __init__(self, c, A, b):
        self.c, self.A, self.b = np.array(c), np.array(A), np.array(b)
        
        if np.any(self.b < 0):
            raise ValueError('The problem is not feasible.')
        
        self.m, self.n = self.A.shape
        
        t = self.m + self.n
        self.L = np.concatenate([np.arange(self.n, t, 1),
                                 np.arange(0, self.n, 1)])
        
        self.T = self.initial_tableau()
        
    def initial_tableau(self):
        I_m = np.eye(self.m)
        A_bar = np.hstack([self.A, I_m])
        c_bar = np.hstack([self.c, np.zeros(self.m)])
        T_1 = np.hstack([0, -c_bar.T, 1])
        T_2 = np.column_stack([self.b, A_bar, np.zeros(self.m)])
        return np.vstack([T_1, T_2])
    
    def pivot_id(self):
        T = self.T.copy()
        column_id = int(np.argwhere(T[0, 1:] < 0)[0]) + 1
                
        if np.all(T[:, column_id] <= 0):
            raise ValueError('The problem is not bounded.')
            
        pos_T = T[:, column_id] <= 0
        T[pos_T, column_id] = np.nan
        row_id = np.nanargmin(T[:, 0] / T[:, column_id])
                
        return row_id, column_id 
    
    def pivot(self):
                
        L, T, m = self.L, self.T, self.m
        row_id, column_id = self.pivot_id()
        
        L_temp = L.copy()
        
        L[row_id - 1] = L_temp[column_id + m - 1]
        L[column_id + m - 1] = L_temp[row_id - 1]
                        
        T[row_id, :] /= T[row_id, column_id]
        
        for i in range(T.shape[0]):
            if i == row_id:
                pass
            else:
                k = -T[i, column_id]
                T[i, :] += k * T[row_id, :]
        
        self.T = T
        self.L = L
        
    def solve(self):
        
        while np.any(self.T[0, :] < 0):
            self.pivot()
                    
        max_val = self.T[0, 0]
        basic = self.L[:self.m]
        nonbasic = self.L[self.m:]
        d1 = dict(zip(nonbasic, np.zeros(self.m)))
        d2 = dict(zip(basic, self.T[1:, 0]))
        return (max_val, d1, d2)      

In [3]:
c = [3, 2]
A = [[1, -1], 
     [3, 1],
     [4, 3]]
b = [2, 5, 7]

In [4]:
linear = Linear_Opt_Problem(c, A, b)
linear.solve()

(5.2,
 {3: 0.0, 4: 0.0},
 {2: 0.5999999999999996, 0: 1.6, 1: 0.19999999999999982})

**Exercise 7**

In [5]:
data = np.load('productMix.npz')
a = data['A']
p = data['p']
m = data['m']
d = data['d']

A = np.row_stack([a, np.eye((4))])
b = np.concatenate([m, d])

In [6]:
Linear_Opt_Problem(p, A, b).solve()

(7453.596491228071,
 {7: 0.0, 4: 0.0, 9: 0.0, 5: 0.0},
 {1: 6.192982456140348,
  3: 1.7894736842105292,
  6: 0.9659649122807016,
  0: 10.0,
  8: 13.807017543859653,
  2: 12.0,
  10: 8.21052631578947})